# 6. 기타 변수 생성

In [1]:
import pandas as pd
import numpy as np
import time
import re

In [2]:
data = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Seoul_location/종로구 실거래(13~18).xlsx")

## 6-1 아파트 브랜드 변수 생성

In [4]:
#아파트 변수 특수문자 제거, 소문자 모두 대문자로 변환
data['아파투'] = ''
for i in range(len(data)):
    text = data.at[i, '아파트']
    data.at[i, '아파투'] = re.sub('[^\w\s]', '', text)
    data.at[i, '아파투'] = data.at[i, '아파투'].upper()

In [5]:
#8대브랜드인지 평가 맞으면 1 아니면 0
string = ['래미안','힐스테이트','롯데캐슬','편한세상','푸르지오','샵','아이파크','IPARK','자이']
data['brand'] = ''

for i in range(len(data)):
    for j in string:
        if j in data.at[i,'아파투']:
            data.at[i,'brand'] = 1

data.at[data['brand'] != 1, 'brand'] = 0
data.drop(['아파투'], axis = 1, inplace=True)

## 6-2 공시지가 변수 생성

In [22]:
# 년도별로 정렬 및 인덱스 초기화
data = data.sort_values(by=['년'], axis=0).reset_index(drop=True)

In [23]:
# 자료형 변경
data['법정동시군구코드'] = data['법정동시군구코드'].astype('str')
data['법정동읍면동코드'] = data['법정동읍면동코드'].astype('str')
data['법정동지번코드'] = data['법정동지번코드'].astype('str')
data['법정동본번코드'] = data['법정동본번코드'].astype('str')
data['법정동부번코드'] = data['법정동부번코드'].astype('str')

In [24]:
# 자리 수 변경
for i in range(len(data)):
    data.at[i,'법정동본번코드'] = data.at[i,'법정동본번코드'].rjust(4, '0')
    data.at[i,'법정동부번코드'] = data.at[i,'법정동부번코드'].rjust(4, '0')

In [25]:
# land_cd, land_price 열 생성
data['land_cd'] = data['법정동시군구코드']+data['법정동읍면동코드']+data['법정동지번코드']+data['법정동본번코드']+data['법정동부번코드']
data['land_price'] = ''

In [71]:
# land_price 매칭함수 생성
def Land_price(x, data):
    A = []

    for i in range(0,6):
        A.append('C:/Users/Yoo/Documents/Capstone/Factor/Land_price/공시지가_201%d_'%(i+3) + str(x) + ".xlsx")

    for i in A:
        print(i)
        Jiga = pd.read_excel("%s"%i)
        Jiga['LAND_CD'] = Jiga['LAND_CD'].astype(str).str.strip()
        C = list(Jiga['LAND_CD'])
        D = list(Jiga['JIGA'])
        
        Num = np.array(data[data.loc[:,'년'] == int(i[55:59])].index.values)
        B = list(data['land_cd'])
        for j in Num:
            if(B[j] in C):
                data.at[j, 'land_price'] = D[C.index(B[j])]
            else:
                continue
    
    return data

In [72]:
# land_price 변수 생성
구 = '종로구'
data = Land_price(구, data)

C:/Users/Yoo/Documents/Capstone/Factor/Land_price/공시지가_2013_강서구.xlsx
C:/Users/Yoo/Documents/Capstone/Factor/Land_price/공시지가_2014_강서구.xlsx
C:/Users/Yoo/Documents/Capstone/Factor/Land_price/공시지가_2015_강서구.xlsx
C:/Users/Yoo/Documents/Capstone/Factor/Land_price/공시지가_2016_강서구.xlsx
C:/Users/Yoo/Documents/Capstone/Factor/Land_price/공시지가_2017_강서구.xlsx
C:/Users/Yoo/Documents/Capstone/Factor/Land_price/공시지가_2018_강서구.xlsx


## 6-3 최고 층 변수 생성

In [3]:
# 아파트명 추출
Apartment = data.아파트.unique()

In [4]:
data['Max_layer'] = ''

In [5]:
# 아파트 변수 생성
for i in Apartment:
    Apart = np.where(data['아파트'].values == i)
    data.loc[Apart[0], 'Max_layer'] = data.층[Apart[0]].max()

## 6-4 행정동별 상하위 동 변수 생성

In [ ]:
data['A_top'] = ''
data['A_bottom'] = ''

In [ ]:
# 거래금액 변수 자료형 변환(문자 -> 숫자)
data.거래금액 = data.거래금액.str.replace(',','').astype(int)

In [ ]:
# 법정동별 거래금액
Area = data.groupby('법정동')[['거래금액']].aggregate("mean")

In [ ]:
# 거래금액 상하위 10% index 추출
Area_top = Area[Area['거래금액'] > np.percentile(Area,90)].index
Area_bottom = Area[Area['거래금액'] < np.percentile(Area,10)].index

In [ ]:
# 상하위 동 변수 생성(상위 10%면 1, 하위 10%면 1)
for i,j in zip(Area_top,Area_bottom):
    top = np.where(data.법정동.values == i)
    bottom = np.where(data.법정동.values == j)
    data.A_top[top[0]] = 1
    data.A_bottom[bottom[0]] = 1

In [ ]:
# 상하위 10% 비해당 동 0 대입
data.loc[data.A_top != 1, 'A_top'] = 0
data.loc[data.A_bottom != 1, 'A_bottom'] = 0

In [6]:
data.to_excel('종로구 실거래(13~18).xlsx', index = False)